In [2]:
# Import packages
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from tensorflow.keras import layers
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [3]:
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [4]:
# Import the pkl file.
with open('ai_image_classifier_small_img.pkl', 'rb') as file:
    recalled_imgs = pickle.load(file)

type(recalled_imgs)

dict

In [5]:
# Set train, test variables.
X_train = recalled_imgs['X_train']
X_test = recalled_imgs['X_test']
y_train = recalled_imgs['y_train']
y_test = recalled_imgs['y_test']

In [6]:
# Convert values to numpy arrays
X_train_aug_np = np.array(X_train)
X_test_np = np.array(X_test)
y_train_aug_np = np.array(y_train) 
y_test_np = np.array(y_test)

In [7]:
# Function for additional layers
# Create function
def nn_cl_bo2(neurons, filter1, filter2, activation, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                 'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                 'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
                 'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons = round(neurons)
    if round(filter1)/32 > 1.5:
        first_layer_filter = 64
    else:
        first_layer_filter = 32
    if round(filter2)/32 > 1.5:
        second_layer_filter = 64
    else:
        second_layer_filter = 32
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    def nn_cl_fun():
        nn = Sequential()
        nn.add(layers.Conv2D(first_layer_filter, (3, 3), activation=activation, input_shape=(64, 64, 3)))
        nn.add(layers.MaxPooling2D((2, 2)))
        if normalization > 0.5:
            nn.add(layers.BatchNormalization())
        for i in range(layers1):
            nn.add(layers.Conv2D(second_layer_filter, (3,3), activation=activation))
            nn.add(layers.MaxPooling2D((2, 2)))
        if dropout > 0.5:
            nn.add(layers.Dropout(dropout_rate, seed=123))
        nn.add(layers.Flatten())
        for i in range(layers2):
            nn.add(layers.Dense(neurons, activation=activation))
        nn.add(layers.Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]})
    score=np.nan_to_num(score)
    score=score.mean()
    return score

In [8]:
params_nn2 ={
    'neurons': (10, 100),
    'filter1': (32, 64),
    'filter2': (32, 64),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3),
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo2, params_nn2, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  filter1  |  filter2  |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
# Read the best parameters.
params_nn_ = nn_bo.max['params']
learning_rate = params_nn_['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]
params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs'] = round(params_nn_['epochs'])
params_nn_['layers1'] = round(params_nn_['layers1'])
params_nn_['layers2'] = round(params_nn_['layers2'])
params_nn_['neurons'] = round(params_nn_['neurons'])
if round(params_nn_['filter1'])/32 > 1.5:
    params_nn_['filter1'] = 64
else:
    params_nn_['filter1'] = 32
if round(params_nn_['filter2'])/32 > 1.5:
    params_nn_['filter2'] = 64
else:
    params_nn_['filter2'] = 32
optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','Adam']
optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
             'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
             'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
             'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
params_nn_['optimizer'] = optimizerD[optimizerL[round(params_nn_['optimizer'])]]
params_nn_

{'activation': 'elu',
 'batch_size': 335,
 'dropout': 0.4360590193711702,
 'dropout_rate': 0.23077874175693686,
 'epochs': 44,
 'filter1': 32,
 'filter2': 32,
 'layers1': 2,
 'layers2': 1,
 'learning_rate': 0.34427962996990447,
 'neurons': 99,
 'normalization': 0.23772645390002278,
 'optimizer': <keras.src.optimizers.sgd.SGD at 0x1b667e6ae00>}